# 1.Librería Request

In [1]:
url = "https://www.officialcharts.com/charts/singles-chart/20240112/7501/"

In [2]:
import requests

In [3]:
html = requests.get(url)

> - Cuando lo ejecutamos sin nigún tipo de atributo, ni argumento ni nada obtenemos un `<Response [200]>`
> - Es un código de tres cifras que nos indica el estado de la petición que estamos haciendo. Si el codigo empieza por 1,2,3 estamos recibiendo un mensaje de "ok". Si empieza por 4 o 5 quiere decir que hay un error

> - HTTP Response **1XX** = el servidor donde nos conectamos continua procesando ya que debe haber algún paso intermedio para completar la request, pero todo va correctamente (no es nada común)
> - HTTP Response **2XX** = indica que la petición ha sido procesada correctamente y la orden se ha ejecutado. Es el mesaje que esperamos y que indica que todo esta ok
> - HTTP Response **3XX** = se trata de mensajes de redirección de la URL de la petición, pero esto también indica que el proceso marcha correctamente
> - HTTP Response **4XX** = la petición no se ha completado porque hay un error del lado del cliente (nosotros). Habitualmente errores en la construcción de la petición o en nuestro user agent. Error 404, indica que estamos intentando acceder a una parte del servidor que no existe.
> - HTTP Response **5XX** = indica que hay un problema del lado del servidor y que esta request no se puede completar por el momento. Error que no queremos ver ni en pintura.



In [4]:
html

<Response [200]>

> - Del atributo `.content` nosotros sacaremos la información que necesitamos

In [5]:
html.content

b'<!DOCTYPE html>\n<html lang="en">\n<head><meta charset="utf-8">\n<title>Official Singles Chart Top 100 on 12&#x2F;1&#x2F;2024 | Official Charts</title>\n<meta name="viewport" content="width=device-width, initial-scale=1">\n<meta name="msapplication-TileColor" content="#f5f2e7">\n<meta name="msapplication-config" content="/favicons/browserconfig.xml">\n<meta name="theme-color" content="#f5f2e7">\n<link rel="apple-touch-icon" sizes="180x180" href="/favicons/apple-touch-icon.png">\n<link rel="icon" type="image/png" sizes="32x32" href="/favicons/favicon-32x32.png">\n<link rel="icon" type="image/png" sizes="16x16" href="/favicons/favicon-16x16.png">\n<link rel="manifest" href="/favicons/site.webmanifest">\n<link rel="mask-icon" color="#0029f3" href="favicons/safari-pinned-tab.svg">\n<link rel="shortcut icon" href="/favicons/favicon.ico">\n<script src="/js/quantcast.js" async="async"></script>\n<script src="https://cdn.tagdeliver.com/cipt/17187.js" async="async"></script>\n<meta name="titl

# 2.robots.txt

> - Define a qué partes podemos acceder dentro de una página web
> - Es como unas puertas que pone la página web para indicar a que aspectos de la página web no se le permite entrar a un usuario
> - Normalmente son partes administrativas de una página web
> - Para acceder a estas puertas basta con escribir  `/robots.txt` en la URL principal de la página web
> - Estas limitaciones no son vinculantes, es decir, si te las saltas, la ley te ampara, no estas cometiendo ningún delito



# 3.BeautifulSoup, DOM y Tags

> - **DOM = Document Object Model**. Es una interfaz para programar documentos web. Se basa en un sistema de tags (etiquetas) las cuales habitualmente se anidan (nestean) como en un diccionario. Estos tags hacen referencia a partes de las páginas web. Son como diccionarios pero que en vez de tener llaves tienen tags.

In [6]:
!pip install bs4
from bs4 import BeautifulSoup

> - Esta libreria nos ayuda a gestionar estos tags
> - Ahora construiremos un objeto soup que nos ayudará a encontrar estos tags dentro del html

In [7]:
soup = BeautifulSoup(html.content)

> - Lo que hace es parsear todos los tags, es decir, estructurar la información de nuestro html

In [8]:
soup

<!DOCTYPE html>
<html lang="en">
<head><meta charset="utf-8"/>
<title>Official Singles Chart Top 100 on 12/1/2024 | Official Charts</title>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<meta content="#f5f2e7" name="msapplication-TileColor"/>
<meta content="/favicons/browserconfig.xml" name="msapplication-config"/>
<meta content="#f5f2e7" name="theme-color"/>
<link href="/favicons/apple-touch-icon.png" rel="apple-touch-icon" sizes="180x180"/>
<link href="/favicons/favicon-32x32.png" rel="icon" sizes="32x32" type="image/png"/>
<link href="/favicons/favicon-16x16.png" rel="icon" sizes="16x16" type="image/png"/>
<link href="/favicons/site.webmanifest" rel="manifest"/>
<link color="#0029f3" href="favicons/safari-pinned-tab.svg" rel="mask-icon"/>
<link href="/favicons/favicon.ico" rel="shortcut icon"/>
<script async="async" src="/js/quantcast.js"></script>
<script async="async" src="https://cdn.tagdeliver.com/cipt/17187.js"></script>
<meta content="Official Singles C

> - Antes de nada lo que tenemos que mirar es como encontrar las etiquetas dentro de este html que reunen la info que nosotros queremos sacar de esa web.
> - Para ello necesitamos realizar una exploración visual dentro de la página web
> - De nuestro objeto soup utilizamos metodos para localizar la información. Método `.find_all()` que lo que hace es aceptar una string que es el tag que nosotros estamos buscando dentro de este html. Puede aceptar como segundo argumento un diccionario en el que indicamos el tipo de tag y la string que estamos buscando.

## 3.1 Localizando la información por etiquetas

> - Definimos las columnas que vamos a necesitar

In [9]:
columnas = ["position", "artist", "title"]

### Localizamos el número que ocupa en la lista

In [10]:
posicion0_10 = [x.text for x in soup.find_all("span", {"class":"digits1 chart-key font-bold"})]
posicion11_99 = [x.text for x in soup.find_all("span", {"class":"digits2 chart-key font-bold"})]
posicion11_100 = [x.text for x in soup.find_all("span", {"class":"digits3 chart-key font-bold"})]
posicion = posicion0_10 + posicion11_99 + posicion11_100
len(posicion)

100

> - Mejoramos el código anterior con un bucle para no repetir código


In [11]:
position = [x.text.replace("Number", "") for digits in range(1, 4) for x in soup.find_all("span", {"class": f"digits{digits} chart-key font-bold"})]
len(position)

100

### Localizamos al artista

> - Método `.replace()` para arreglar los nombres
> - Método `.title()` para que solo la primera sea mayúscula

In [12]:
artist = [x.text.replace("/", " & ").title() for x in soup.find_all("a", {"class":"chart-artist text-lg inline-block"})]

In [13]:
len(artist)

100

### Localizamos el título de la canción

In [14]:
title = [x.text.replace("New", "").title() for x in soup.find_all("a", {"class":"chart-name font-bold inline-block"})]

In [15]:
len(title)

100

### Creamos el DataFrame a partir de un diccionario

In [16]:
import pandas as pd
dct = dict ([("position", position), ("artist", artist), ("title", title)])
df = pd.DataFrame(dct)
df = df.set_index("position") # Ponemos como índdice la posición en la lista
df.head(10)

,artist,title
position,,
1,Noah Kahan,Stick Season
2,Sophie Ellis-Bextor,Murder On The Dancefloor
3,Jack Harlow,Lovin On Me
4,Casso & Raye & D-Block Europe,Prada
5,Tate Mcrae,Greedy
6,Teddy Swims,Lose Control
7,Taylor Swift,Cruel Summer
8,Tyla,Water
9,Dua Lipa,Houdini


## 3.2 Table Rows

> - Puede ser que en otras páginas web no se siga el DOM y no este bien estructurado el código HTML, lo cual puede dificultar mucho la extracción de los datos
> - Estos table rows contienen toda la información de una "fila" y por ello vamos a iterar sobre ellos
> - Método muy útil en caso de que la extracción de info por etiquetas se nos pueda complicar
> - *Justo en esta web no tenemos **table rows (tr)**, cogeremos el div que tiene todo el contenido de cada uno de los elementos e iteraremos sobre él*

In [17]:
data = [x.text for x in soup.find_all("div", {"class":"chart-item-content relative flex w-full"})]
data[0]

'Number 1STICK SEASONNOAH KAHANLW: 1, Peak: 1, Weeks: 15'

> - Con el código que se muestra a continuación, transformamos los números que marcan la posición en integers para poder realizar una separación más adelante

In [18]:
def clean_rows(lista):

  new_list = []

  for row in lista:
    try:
      int(row[7])
      new_list.append(row)

    except:
      pass

  return new_list

In [19]:
rows = clean_rows(data)

> - Función filter para filtar una lista y que nos devuelva solo los elementos que cumplan una condición que nosotros imponemos. `filter(condicion, iterable)`



# 4.Contruyendo un Web Spider

In [20]:
import pandas as pd

In [21]:
dct = {
    "position" : position,
    "artist" : artist,
    "title" : title
}

df = pd.DataFrame(dct)
df = df.set_index("position") # Ponemos como índice la posición en la lista
df["chart"] = "UK" # Incluimos el páis en la que se ha obtenido la lista

> - Buscamos la fecha para identificar al DataFrame. En este caso la lista se divide por semanas

In [22]:
import datetime

In [23]:
semana = datetime.datetime.today().strftime("%V") #Para saber en que semana del año estamos

In [24]:
df["week"] = int(semana)
df.head()

,artist,title,chart,week
position,,,,
1,Noah Kahan,Stick Season,UK,4
2,Sophie Ellis-Bextor,Murder On The Dancefloor,UK,4
3,Jack Harlow,Lovin On Me,UK,4
4,Casso & Raye & D-Block Europe,Prada,UK,4
5,Tate Mcrae,Greedy,UK,4


## 4.1 Construyendo el spider

In [25]:
def listas_uk(url):
  html = requests.get(url)
  soup = BeautifulSoup(html.content)

  columnas = ["position", "artist", "title"]

  position = [x.text.replace("Number", "") for digits in range(1, 4) for x in soup.find_all("span", {"class": f"digits{digits} chart-key font-bold"})]
  artist = [x.text.replace("/", " & ").title() for x in soup.find_all("a", {"class":"chart-artist text-lg inline-block"})]
  title = [x.text.replace("New", "").title() for x in soup.find_all("a", {"class":"chart-name font-bold inline-block"})]

  dct = {
    "position" : position,
    "artist" : artist,
    "title" : title
    }

  df = pd.DataFrame(dct)
  df = df.set_index("position") # Ponemos como índice la posición en la lista

  df["chart"] = "UK" # Incluimos el páis en la que se ha obtenido la lista

  semana = datetime.datetime.today().strftime("%V")  #Para saber en que semana del año estamos
  df["week"] = int(semana)

  return df

In [26]:
nuevos_datos = listas_uk(url)

In [27]:
nuevos_datos

,artist,title,chart,week
position,,,,
1,Noah Kahan,Stick Season,UK,4
2,Sophie Ellis-Bextor,Murder On The Dancefloor,UK,4
3,Jack Harlow,Lovin On Me,UK,4
4,Casso & Raye & D-Block Europe,Prada,UK,4
5,Tate Mcrae,Greedy,UK,4
...,...,...,...,...
96,Whethan & Elley Duhe,Money On The Dash,UK,4
97,Goo Goo Dolls,Iris,UK,4
98,Olivia Rodrigo,Reall-American Bitch,UK,4


In [28]:
nuevos_datos["artist"].value_counts()

Noah Kahan                     3
Olivia Rodrigo                 3
Lewis Capaldi                  3
Taylor Swift                   3
Dua Lipa                       2
                              ..
Eliza Rose & Calvin Harris     1
Nicki Minaj Ft Lil Uzi Vert    1
Kanye West Ft Pusha T          1
Paul Russell                   1
Zach Bryan                     1
Name: artist, Length: 86, dtype: int64

## 4.2 Acceso a URLS para obtener listas de otras fechas


In [29]:
url = "https://www.officialcharts.com/charts/singles-chart/20240112/7501/"

fecha = "20240112"

In [30]:
fecha

'20240112'

> - Parseamos/transformamos el objeto con la fecha que es de tipo string a formato **dateTime**

In [31]:
fecha_obj = datetime.datetime.strptime(fecha, "%Y%m%d")

fecha_obj

datetime.datetime(2024, 1, 12, 0, 0)

> - `datetime.timedelta` para realizar variaciones en las fechas

In [32]:
fecha_nueva = fecha_obj - datetime.timedelta(days=7)

> - Lo volvemos a transformar a formato string para poder modificar la URL de forma iterativa

In [33]:
fecha_nueva.strftime("%Y%m%d")

'20240105'

In [34]:
def restar_semanas(fecha_string, n_semanas_restar):

  fecha_obj = datetime.datetime.strptime(fecha_string, "%Y%m%d")

  fecha_nueva = fecha_obj - datetime.timedelta(weeks=n_semanas_restar)

  return fecha_nueva.strftime("%Y%m%d")

In [35]:
fecha

'20240112'

In [36]:
restar_semanas(fecha, n_semanas_restar = 2)

'20231229'

### 4.2.1 Cambiar URL

In [37]:
url_nueva= "https://www.officialcharts.com/charts/singles-chart/" + fecha + "/7501/"

## 4.3 Web Spider múltiple

In [38]:
fecha_actual =  "20240112"

def spider_listas(fecha_actual, n_semanas_restar):

  df_list = []

  for i in range(n_semanas_restar):

    nueva_fecha = restar_semanas(fecha_actual, i)

    url_nueva= "https://www.officialcharts.com/charts/singles-chart/" + nueva_fecha + "/7501/"

    df = listas_uk(url_nueva)

    semana = datetime.datetime.strptime(nueva_fecha, "%Y%m%d").strftime("%V")

    df["week"] = semana

    df_list.append(df)

  return pd.concat(df_list)

In [39]:
df = spider_listas(fecha, 5)

In [40]:
df

,artist,title,chart,week
position,,,,
1,Noah Kahan,Stick Season,UK,02
2,Sophie Ellis-Bextor,Murder On The Dancefloor,UK,02
3,Jack Harlow,Lovin On Me,UK,02
4,Casso & Raye & D-Block Europe,Prada,UK,02
5,Tate Mcrae,Greedy,UK,02
...,...,...,...,...
96,Becky Hill & Chase & Status,Disconnect,UK,50
97,Jung Kook,Standing Next To You,UK,50
98,Fred Again & Jozzy,Reten,UK,50


In [41]:
df["artist"].value_counts()

Taylor Swift           15
Tate Mcrae             13
Noah Kahan             12
Olivia Rodrigo         11
Nicki Minaj             7
                       ..
Tears For Fears         1
Drake                   1
Nathan Evans            1
Natasha Bedingfield     1
Drake Ft Yeat           1
Name: artist, Length: 168, dtype: int64